# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lebu,-37.6167,-73.6500,13.83,83,20,6.80,CL,1742168054
1,1,ust-nera,64.5667,143.2000,-27.12,91,57,1.29,RU,1742168055
2,2,ribeira grande,38.5167,-28.7000,15.22,72,40,7.72,PT,1742168058
3,3,ushuaia,-54.8000,-68.3000,12.81,58,75,2.57,AR,1742168031
4,4,longyearbyen,78.2186,15.6401,-19.09,65,40,2.57,SJ,1742168060


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.scatter(
    x='Lng', 
    y='Lat', 
    size='Humidity', 
    color='Humidity', 
    colormap='Viridis', 
    hover_cols=['City', 'Country', 'Humidity'],
    title='City Humidity Map'
)

# Display the map
humidity_map

:Scatter   [Lng]   (Lat,Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,wailua homesteads,22.0669,-159.3780,25.63,79,0,4.12,US,1742167983
139,139,remire-montjoly,4.9167,-52.2667,26.02,89,0,2.57,GF,1742168239
143,143,al kharijah,25.4514,30.5464,23.79,28,0,3.59,EG,1742168246
153,153,lihue,21.9789,-159.3672,25.97,78,0,4.12,US,1742168257
195,195,lohardaga,23.4333,84.6833,23.01,38,0,2.08,IN,1742168309


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
15,wailua homesteads,US,22.0669,-159.3780,79,
139,remire-montjoly,GF,4.9167,-52.2667,89,
143,al kharijah,EG,25.4514,30.5464,28,
153,lihue,US,21.9789,-159.3672,78,
195,lohardaga,IN,23.4333,84.6833,38,
227,kapa'a,US,22.0752,-159.3190,78,
316,san patricio,US,28.0170,-97.5169,23,
377,thasra,IN,22.8000,73.2167,22,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 10000
api_key = "geoapify_key"

params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": api_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wailua homesteads - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
al kharijah - nearest hotel: No hotel found
lihue - nearest hotel: No hotel found
lohardaga - nearest hotel: No hotel found
kapa'a - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
thasra - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
15,wailua homesteads,US,22.0669,-159.3780,79,No hotel found
139,remire-montjoly,GF,4.9167,-52.2667,89,No hotel found
143,al kharijah,EG,25.4514,30.5464,28,No hotel found
153,lihue,US,21.9789,-159.3672,78,No hotel found
195,lohardaga,IN,23.4333,84.6833,38,No hotel found
227,kapa'a,US,22.0752,-159.3190,78,No hotel found
316,san patricio,US,28.0170,-97.5169,23,No hotel found
377,thasra,IN,22.8000,73.2167,22,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [32]:
%%capture --no-display

# Configure the map plot using a scatter plot
hotel_map = hotel_df.hvplot.scatter(
    x='Lng', 
    y='Lat', 
    size='Humidity', 
    color='Humidity', 
    colormap='Viridis', 
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'],
    title='Hotel Locations with Humidity'
)

# Display the map
hotel_map

:Scatter   [Lng]   (Lat,Humidity,City,Country,Hotel Name)